---

# Chapter 17: Data Management in Distributed Systems (CQRS, Event Sourcing, Saga, Sharding)

## Opening Context

In a monolithic application, data management is relatively straightforward: a single database handles all reads and writes, transactions ensure consistency, and queries are simple. However, when we decompose a system into microservices, each service often owns its own database. This **database‑per‑service** pattern brings new challenges:

- How do we maintain consistency across services without distributed transactions?
- How do we handle complex queries that span multiple services?
- How do we scale data storage and access under high load?
- How do we maintain an audit trail of changes?

This chapter addresses these challenges with four powerful patterns:

1. **CQRS (Command Query Responsibility Segregation)** – Separates the models for writing (commands) and reading (queries) to optimize each independently.
2. **Event Sourcing** – Stores the state of an entity as a sequence of immutable events, enabling auditability, time travel, and rebuilding of state.
3. **Saga Pattern** – Manages distributed transactions across multiple services using a sequence of local transactions and compensating actions.
4. **Sharding and Partitioning** – Distributes data across multiple databases or nodes to achieve scalability and performance.

These patterns are essential building blocks for modern, scalable, and resilient distributed systems. By the end of this chapter, you’ll understand how to apply them to real‑world scenarios.

---

## 17.1 CQRS (Command Query Responsibility Segregation)

### Intent
*Separate the operations that modify data (commands) from those that retrieve data (queries) into different models, allowing each to be optimized, scaled, and evolved independently.*

### The Problem

In traditional CRUD (Create, Read, Update, Delete) systems, a single model is used for both reads and writes. This works well for simple applications, but as complexity grows, several issues arise:

- **Impedance mismatch** – The object‑oriented domain model used for writes may not match the shape of data needed for complex read queries.
- **Performance conflicts** – Write‑optimized storage (e.g., normalized relational tables) may be inefficient for complex read queries, requiring joins or denormalization that slow down writes.
- **Scalability** – Reads often outnumber writes by orders of magnitude. Scaling both on the same model is inefficient; you might need to scale reads independently.
- **Security** – Read and write operations may have different security requirements, which are harder to enforce in a unified model.

### The Solution: CQRS

CQRS separates the **command side** (handling updates) from the **query side** (handling reads). Each side can have its own data store, optimized for its purpose. Commands use a domain model (often with complex business logic), while queries use a simplified read model (denormalized, possibly cached). The two models are kept eventually consistent, often through events.

```
[Client] --> Command --> Command Handler --> Domain Model --> Write DB
                     ↘
                      Event → Read Model Updater → Read DB
[Client] --> Query --> Query Handler ----------------------> Read DB
```

#### Simple CQRS Example (Without Event Sourcing)

Let’s build a simple order management system with separate write and read models. We'll use a relational database for writes (normalized) and a document store or cache for reads (denormalized).

##### Write Model (Command Side)

```typescript
// domain/order.ts
export class Order {
  constructor(
    public readonly id: string,
    public customerId: string,
    public items: OrderItem[],
    public status: OrderStatus,
    public createdAt: Date
  ) {}

  addItem(productId: string, quantity: number, price: number): void {
    this.items.push({ productId, quantity, price });
  }

  cancel(): void {
    if (this.status === 'shipped') throw new Error('Cannot cancel shipped order');
    this.status = 'cancelled';
  }
}

// command/place-order.command.ts
export class PlaceOrderCommand {
  constructor(
    public readonly customerId: string,
    public readonly items: Array<{ productId: string; quantity: number; price: number }>
  ) {}
}

// command/order.command-handler.ts
import { Order } from '../domain/order';
import { OrderRepository } from '../repositories/order.repository'; // write repository

export class OrderCommandHandler {
  constructor(private orderRepository: OrderRepository) {}

  async handle(command: PlaceOrderCommand): Promise<string> {
    // Create order aggregate
    const order = new Order(
      generateUuid(),
      command.customerId,
      command.items,
      'pending',
      new Date()
    );

    // Save to write database
    await this.orderRepository.save(order);

    // Publish event (e.g., OrderPlaced) for read model update
    await this.publishEvent('OrderPlaced', {
      orderId: order.id,
      customerId: order.customerId,
      items: order.items,
      total: order.items.reduce((sum, i) => sum + i.price * i.quantity, 0),
      createdAt: order.createdAt
    });

    return order.id;
  }

  private async publishEvent(eventType: string, data: any): Promise<void> {
    // In real system, publish to message broker (Kafka, RabbitMQ)
    console.log(`Event published: ${eventType}`, data);
  }
}
```

##### Write Repository (using SQL database)

```typescript
// repositories/order.repository.ts
import { Order } from '../domain/order';

export interface OrderRepository {
  save(order: Order): Promise<void>;
  findById(id: string): Promise<Order | null>;
}

export class SqlOrderRepository implements OrderRepository {
  async save(order: Order): Promise<void> {
    // Serialize order to relational tables (orders, order_items)
    // Use SQL INSERT/UPDATE
  }
  async findById(id: string): Promise<Order | null> {
    // Query and reconstruct Order object
  }
}
```

##### Read Model (Query Side)

The read model is denormalized for fast querying, possibly stored in a separate database (e.g., MongoDB, Elasticsearch, Redis).

```typescript
// read-models/order-view.ts
export interface OrderView {
  orderId: string;
  customerId: string;
  customerName?: string; // denormalized from customer service
  items: Array<{ productId: string; productName: string; quantity: number; price: number }>;
  total: number;
  status: string;
  createdAt: Date;
}

// read-repositories/order-view.repository.ts
export interface OrderViewRepository {
  save(view: OrderView): Promise<void>;
  findById(orderId: string): Promise<OrderView | null>;
  findByCustomer(customerId: string): Promise<OrderView[]>;
}

// Event handlers to update read model
// event-handlers/order-placed.handler.ts
import { OrderViewRepository } from '../read-repositories/order-view.repository';

export class OrderPlacedHandler {
  constructor(private orderViewRepository: OrderViewRepository) {}

  async handle(event: any): Promise<void> {
    const orderView: OrderView = {
      orderId: event.orderId,
      customerId: event.customerId,
      // we might need to enrich with product names and customer name by calling other services
      items: event.items, // simplistic
      total: event.total,
      status: 'pending',
      createdAt: event.createdAt
    };
    await this.orderViewRepository.save(orderView);
  }
}
```

##### Query API

```typescript
// controllers/order.controller.ts (query side)
import { OrderViewRepository } from '../read-repositories/order-view.repository';

export class OrderQueryController {
  constructor(private orderViewRepository: OrderViewRepository) {}

  async getOrder(req: Request, res: Response): Promise<void> {
    const orderId = req.params.id;
    const order = await this.orderViewRepository.findById(orderId);
    if (!order) {
      res.status(404).json({ error: 'Order not found' });
      return;
    }
    res.json(order);
  }

  async getCustomerOrders(req: Request, res: Response): Promise<void> {
    const customerId = req.params.customerId;
    const orders = await this.orderViewRepository.findByCustomer(customerId);
    res.json(orders);
  }
}
```

**Explanation**:
- The command side uses a domain model with business logic and saves to a normalized write database.
- The command handler publishes an event after successful write.
- An event handler (separate component) updates the denormalized read model.
- The query side reads directly from the read model, which is optimized for queries (e.g., pre‑joined, aggregated).
- The read model is **eventually consistent** with the write side; there is a small delay.

### When to Use CQRS

CQRS adds complexity, so it should be used when:
- The application has significantly different read and write workloads (e.g., reads >> writes).
- Complex queries are hard to perform on the write model.
- You need to scale reads and writes independently.
- Different teams will work on read and write sides (e.g., separate bounded contexts).

### Benefits

- **Scalability** – Read and write stores can be scaled independently.
- **Optimized performance** – Each model is tuned for its purpose (e.g., denormalized reads, normalized writes).
- **Security** – Can enforce different permissions for reads and writes.
- **Flexibility** – You can use different databases for reads (e.g., Elasticsearch) and writes (e.g., PostgreSQL).

### Drawbacks

- **Complexity** – More moving parts, eventual consistency, and potential for data staleness.
- **Eventual consistency** – Some users may see outdated data; not suitable for all use cases.
- **Higher development effort** – Requires building event propagation and synchronization.

---

## 17.2 Event Sourcing

### Intent
*Store the state of an entity as a sequence of state‑changing events. Instead of storing the current state, the system stores all events that have happened; the current state can be derived (replayed) from these events.*

### The Problem

Traditional persistence stores the current state of an entity, overwriting previous states. This has limitations:
- **No audit trail** – You lose historical changes.
- **No time travel** – You cannot easily see the state at a point in the past.
- **Complex event‑driven integrations** – You often emit events manually, which may be lost if not persisted.
- **Debugging difficulties** – When a bug occurs, you cannot replay the sequence of events that led to the faulty state.

### The Solution: Event Sourcing

With Event Sourcing, every change to an entity is captured as an immutable event. The current state is obtained by replaying all events for that entity. The event store is the primary source of truth; the current state can be cached for performance.

#### Core Concepts

- **Event** – An immutable record of something that happened (e.g., `OrderPlaced`, `OrderItemAdded`, `OrderCancelled`).
- **Aggregate** – A cluster of domain objects that can be treated as a single unit (e.g., an `Order` aggregate). Events are applied to the aggregate to change its state.
- **Event Store** – A database that stores events, typically append‑only. It also acts as a message queue for event subscribers.
- **Projection** – A read model built by processing events. Projections are used for queries and can be stored in separate databases.

#### Example: Event‑Sourced Order Aggregate

Let's implement a simple event‑sourced order aggregate.

```typescript
// events/order.events.ts
export interface DomainEvent {
  aggregateId: string;
  type: string;
  timestamp: Date;
}

export class OrderPlacedEvent implements DomainEvent {
  constructor(
    public readonly aggregateId: string,
    public readonly customerId: string,
    public readonly items: OrderItem[],
    public readonly timestamp: Date = new Date()
  ) {}
  readonly type = 'OrderPlaced';
}

export class OrderItemAddedEvent implements DomainEvent {
  constructor(
    public readonly aggregateId: string,
    public readonly item: OrderItem,
    public readonly timestamp: Date = new Date()
  ) {}
  readonly type = 'OrderItemAdded';
}

export class OrderCancelledEvent implements DomainEvent {
  constructor(
    public readonly aggregateId: string,
    public readonly reason: string,
    public readonly timestamp: Date = new Date()
  ) {}
  readonly type = 'OrderCancelled';
}
```

```typescript
// domain/order.aggregate.ts
import { DomainEvent } from '../events';

export class OrderAggregate {
  private id: string;
  private customerId: string;
  private items: OrderItem[] = [];
  private status: 'pending' | 'cancelled' = 'pending';
  private version: number = 0; // for concurrency control

  // Rebuild state from events
  static loadFromHistory(events: DomainEvent[]): OrderAggregate {
    const order = new OrderAggregate();
    events.forEach(event => order.apply(event, true));
    return order;
  }

  // Command: place order
  placeOrder(customerId: string, items: OrderItem[]): DomainEvent[] {
    if (this.id) throw new Error('Order already exists');
    const event = new OrderPlacedEvent(generateUuid(), customerId, items);
    this.apply(event);
    return [event];
  }

  // Command: add item
  addItem(item: OrderItem): DomainEvent[] {
    if (this.status !== 'pending') throw new Error('Cannot add item to cancelled order');
    const event = new OrderItemAddedEvent(this.id, item);
    this.apply(event);
    return [event];
  }

  // Command: cancel
  cancel(reason: string): DomainEvent[] {
    if (this.status === 'cancelled') throw new Error('Already cancelled');
    const event = new OrderCancelledEvent(this.id, reason);
    this.apply(event);
    return [event];
  }

  // Apply event to mutate state (and optionally validate)
  private apply(event: DomainEvent, isReplay: boolean = false): void {
    if (!isReplay) {
      // In real system, we might publish event after storing
    }
    switch (event.type) {
      case 'OrderPlaced':
        const e = event as OrderPlacedEvent;
        this.id = e.aggregateId;
        this.customerId = e.customerId;
        this.items = e.items;
        break;
      case 'OrderItemAdded':
        const e2 = event as OrderItemAddedEvent;
        this.items.push(e2.item);
        break;
      case 'OrderCancelled':
        this.status = 'cancelled';
        break;
    }
    this.version++;
  }
}
```

```typescript
// event-store/event-store.ts
export interface EventStore {
  saveEvents(aggregateId: string, events: DomainEvent[], expectedVersion: number): Promise<void>;
  getEvents(aggregateId: string): Promise<DomainEvent[]>;
}

// In-memory implementation for illustration
export class InMemoryEventStore implements EventStore {
  private store: Map<string, DomainEvent[]> = new Map();

  async saveEvents(aggregateId: string, events: DomainEvent[], expectedVersion: number): Promise<void> {
    const current = this.store.get(aggregateId) || [];
    if (current.length !== expectedVersion) {
      throw new Error('Concurrency conflict');
    }
    this.store.set(aggregateId, [...current, ...events]);
  }

  async getEvents(aggregateId: string): Promise<DomainEvent[]> {
    return this.store.get(aggregateId) || [];
  }
}
```

```typescript
// repositories/order.repository.ts (event-sourced)
import { OrderAggregate } from '../domain/order.aggregate';
import { EventStore } from '../event-store/event-store';

export class OrderRepository {
  constructor(private eventStore: EventStore) {}

  async save(aggregate: OrderAggregate): Promise<void> {
    // Assuming aggregate tracks uncommitted changes; for simplicity we don't show that here
    // In real implementation, aggregate would have getUncommittedEvents() and clearEvents()
  }

  async findById(id: string): Promise<OrderAggregate | null> {
    const events = await this.eventStore.getEvents(id);
    if (events.length === 0) return null;
    return OrderAggregate.loadFromHistory(events);
  }
}
```

**Explanation**:
- The `OrderAggregate` processes commands and produces events. It applies events to mutate its internal state.
- The `apply` method is used both when handling commands (to update state) and when rebuilding from history.
- The event store saves events in an append‑only fashion. It uses optimistic concurrency (`expectedVersion`) to prevent conflicts.
- To retrieve an aggregate, we load all its events and replay them.

#### Building Read Models with Projections

Event‑sourced systems use **projections** to build read models. A projection subscribes to events and updates query‑friendly views.

```typescript
// projections/order-list.projection.ts
import { OrderViewRepository } from '../read-repositories/order-view.repository';

export class OrderListProjection {
  constructor(private orderViewRepository: OrderViewRepository) {}

  async onOrderPlaced(event: OrderPlacedEvent): Promise<void> {
    await this.orderViewRepository.save({
      orderId: event.aggregateId,
      customerId: event.customerId,
      items: event.items.map(i => ({ ...i, productName: '' })), // could enrich
      total: event.items.reduce((sum, i) => sum + i.price * i.quantity, 0),
      status: 'pending',
      createdAt: event.timestamp
    });
  }

  async onOrderCancelled(event: OrderCancelledEvent): Promise<void> {
    const view = await this.orderViewRepository.findById(event.aggregateId);
    if (view) {
      view.status = 'cancelled';
      await this.orderViewRepository.save(view);
    }
  }
}
```

**Explanation**:
- Projections are event handlers that update denormalized read models.
- They can be rebuilt by replaying all events from the beginning (if the projection logic changes).

### Benefits of Event Sourcing

- **Complete audit log** – Every change is recorded, providing a perfect audit trail.
- **Time travel** – You can reconstruct the state at any point in time.
- **Debugging and analysis** – Reproduce bugs by replaying events.
- **Flexible projections** – Build any number of read models without impacting the write side.
- **Event‑driven architecture** – Events can be published to other services naturally.

### Drawbacks

- **Complexity** – Learning curve, event versioning, and handling schema evolution.
- **Storage** – Event stores can grow large; snapshots are often used to limit replay.
- **Eventual consistency** – Read models are eventually consistent.
- **Querying the event store** – Direct querying of events is limited; you rely on projections.

### Snapshotting

To avoid replaying all events from the beginning, you can periodically take a **snapshot** of the aggregate's state. When loading, you load the latest snapshot and then apply events after that snapshot.

```typescript
// Simplified snapshotting
class OrderAggregate {
  static loadFromSnapshot(snapshot: OrderSnapshot, subsequentEvents: DomainEvent[]): OrderAggregate {
    const order = new OrderAggregate();
    // restore from snapshot...
    subsequentEvents.forEach(e => order.apply(e, true));
    return order;
  }
}
```

---

## 17.3 Saga Pattern

### Intent
*Manage distributed transactions across multiple services by using a sequence of local transactions, each with a compensating action that undoes it if subsequent steps fail.*

### The Problem

In a distributed system, a single business operation often spans multiple services. For example, placing an order might involve:
1. Order Service: create order (status = pending)
2. Payment Service: reserve credit
3. Inventory Service: reserve items
4. Order Service: confirm order (status = confirmed)

If any step fails, we need to undo the previous steps to maintain consistency. Traditional ACID transactions are not possible across different databases; we need a different approach.

### The Solution: Saga

A **saga** is a sequence of local transactions. Each local transaction updates the database and publishes a message or event. If a local transaction fails, the saga executes **compensating transactions** that undo the changes made by the preceding local transactions.

There are two common coordination styles:

- **Choreography** – Each service produces and listens to events, deciding when to act or compensate.
- **Orchestration** – A central coordinator (saga orchestrator) tells each service what to do and handles compensation.

#### Example: Choreography‑Based Saga

Let's model an order placement saga using events.

**Step 1: Order Service creates order and publishes `OrderCreated`**

```typescript
// order-service/commands/place-order.ts
async function placeOrder(customerId: string, items: any[]) {
  const order = createOrder(customerId, items, 'PENDING');
  await saveOrder(order);
  await publishEvent('OrderCreated', { orderId: order.id, customerId, items });
  return order.id;
}
```

**Step 2: Payment Service listens, reserves credit, publishes `PaymentReserved` or `PaymentFailed`**

```typescript
// payment-service/events/order-created.handler.ts
async function handleOrderCreated(event: any) {
  try {
    await reserveCredit(event.customerId, calculateTotal(event.items));
    await publishEvent('PaymentReserved', { orderId: event.orderId });
  } catch (err) {
    await publishEvent('PaymentFailed', { orderId: event.orderId, reason: err.message });
  }
}
```

**Step 3: Inventory Service listens for `PaymentReserved`, reserves items, publishes `InventoryReserved` or `InventoryFailed`**

```typescript
// inventory-service/events/payment-reserved.handler.ts
async function handlePaymentReserved(event: any) {
  try {
    for (const item of event.items) {
      await reserveItem(item.productId, item.quantity);
    }
    await publishEvent('InventoryReserved', { orderId: event.orderId });
  } catch (err) {
    await publishEvent('InventoryFailed', { orderId: event.orderId, reason: err.message });
  }
}
```

**Step 4: Order Service listens for `InventoryReserved` and confirms order**

```typescript
// order-service/events/inventory-reserved.handler.ts
async function handleInventoryReserved(event: any) {
  await updateOrderStatus(event.orderId, 'CONFIRMED');
  await publishEvent('OrderConfirmed', { orderId: event.orderId });
}
```

**Compensation: If `PaymentFailed` occurs, Order Service cancels the order**

```typescript
// order-service/events/payment-failed.handler.ts
async function handlePaymentFailed(event: any) {
  await updateOrderStatus(event.orderId, 'CANCELLED');
  // maybe also notify customer
}
```

**If `InventoryFailed` occurs, Order Service cancels and Payment Service releases credit (by listening to `InventoryFailed`)**

```typescript
// payment-service/events/inventory-failed.handler.ts
async function handleInventoryFailed(event: any) {
  await releaseCredit(event.orderId); // compensating action
}
```

**Explanation**:
- Each service reacts to events and performs its local transaction.
- If a step fails, an event is published that triggers compensating actions in previous services.
- This is **choreography** – no central controller.

#### Example: Orchestration‑Based Saga

Now let's implement an orchestrator that coordinates the saga.

```typescript
// saga-orchestrator/order-saga.ts
export class OrderSaga {
  async execute(orderId: string, customerId: string, items: any[]): Promise<void> {
    try {
      // Step 1: Create order
      await this.callOrderService('create', { orderId, customerId, items });
      
      // Step 2: Reserve payment
      await this.callPaymentService('reserve', { orderId, customerId, amount: total(items) });
      
      // Step 3: Reserve inventory
      await this.callInventoryService('reserve', { orderId, items });
      
      // Step 4: Confirm order
      await this.callOrderService('confirm', { orderId });
    } catch (error) {
      // Compensate
      await this.compensate(orderId);
      throw error;
    }
  }

  private async compensate(orderId: string): Promise<void> {
    // Call compensating actions in reverse order
    await this.callInventoryService('release', { orderId }); // if supported
    await this.callPaymentService('release', { orderId });
    await this.callOrderService('cancel', { orderId });
  }

  // Methods to call external services (using HTTP, messaging, etc.)
}
```

**Explanation**:
- The orchestrator explicitly calls each service in order.
- If any call fails, it triggers compensating calls in reverse order.
- This centralises the saga logic, making it easier to track and manage.

### Saga Trade‑offs

- **Complexity** – Managing compensating logic and eventual consistency is harder than ACID.
- **Idempotency** – Services must handle duplicate requests (e.g., due to retries).
- **Observability** – You need to track saga states and handle failures.

Saga is essential for maintaining data consistency in microservices without distributed transactions.

---

## 17.4 Sharding and Partitioning Patterns

### Intent
*Distribute data across multiple databases or nodes to achieve scalability, performance, and high availability.*

### The Problem

A single database has limits: storage capacity, connection limits, write throughput, and query performance. As data grows, these limits become bottlenecks. We need a way to split data across multiple physical databases while presenting a logical view to the application.

### The Solution: Sharding

**Sharding** (also called horizontal partitioning) splits data across multiple databases (shards) based on a **shard key**. Each shard holds a subset of the data. The application routes queries to the appropriate shard(s).

#### Key Concepts

- **Shard Key** – A field or combination of fields used to determine which shard holds a given record (e.g., `customer_id`, `order_id`).
- **Sharding Strategy** – How the shard key maps to a shard (e.g., range‑based, hash‑based, directory‑based).
- **Cross‑Shard Queries** – Queries that need data from multiple shards are complex and often avoided; they require scatter‑gather.

#### Sharding Strategies

##### 1. Range‑Based Sharding

Partition data by ranges of the shard key, e.g., customers with IDs 1–1000 in shard 1, 1001–2000 in shard 2, etc.

**Pros**:
- Easy to implement.
- Range queries on the shard key are efficient within a shard.

**Cons**:
- Hotspots if ranges are not balanced (e.g., new customers all go to the last shard).
- Resharding (splitting a range) can be complex.

##### 2. Hash‑Based Sharding

Compute a hash of the shard key and mod by the number of shards: `shard = hash(key) % N`.

**Pros**:
- Even distribution of data.
- Simple to route.

**Cons**:
- Resharding (adding/removing shards) requires rehashing, often needing data migration.
- Range queries across shards are inefficient.

##### 3. Directory‑Based Sharding

Maintain a lookup table that maps each key to a shard.

**Pros**:
- Flexible; you can move data easily.
- Supports custom sharding logic.

**Cons**:
- Lookup table can become a bottleneck or single point of failure.
- Additional lookup overhead.

#### Implementing a Simple Sharded Repository

Assume we have two shards (PostgreSQL instances). We'll use hash‑based sharding on `customerId`.

```typescript
// sharding/shard-manager.ts
import { Client } from 'pg';

export class ShardManager {
  private shards: Client[] = [];

  constructor(shardConfigs: any[]) {
    for (const config of shardConfigs) {
      this.shards.push(new Client(config));
    }
  }

  getShardForKey(key: string): Client {
    const hash = this.hash(key);
    const shardIndex = hash % this.shards.length;
    return this.shards[shardIndex];
  }

  private hash(key: string): number {
    // Simple djb2 hash for demonstration
    let hash = 5381;
    for (let i = 0; i < key.length; i++) {
      hash = (hash << 5) + hash + key.charCodeAt(i);
    }
    return Math.abs(hash);
  }

  async query(key: string, sql: string, params?: any[]): Promise<any> {
    const client = this.getShardForKey(key);
    return client.query(sql, params);
  }
}
```

**Usage in Repository**:

```typescript
// repositories/order.repository.ts (sharded)
export class ShardedOrderRepository {
  constructor(private shardManager: ShardManager) {}

  async save(order: Order): Promise<void> {
    // Use order.customerId as shard key
    await this.shardManager.query(
      order.customerId,
      'INSERT INTO orders (id, customer_id, total, status) VALUES ($1, $2, $3, $4)',
      [order.id, order.customerId, order.total, order.status]
    );
  }

  async findByCustomer(customerId: string): Promise<Order[]> {
    const result = await this.shardManager.query(
      customerId,
      'SELECT * FROM orders WHERE customer_id = $1',
      [customerId]
    );
    return result.rows;
  }

  // This query requires scatter-gather because orderId alone doesn't give shard
  async findById(orderId: string): Promise<Order | null> {
    // We need to know customerId to locate shard, or we could shard by orderId.
    // If sharded by orderId, we can route directly.
    // Here we assume we don't have customerId, so we must query all shards.
    const queries = this.shardManager.shards.map(shard => 
      shard.query('SELECT * FROM orders WHERE id = $1', [orderId])
    );
    const results = await Promise.all(queries);
    for (const result of results) {
      if (result.rows.length > 0) return result.rows[0];
    }
    return null;
  }
}
```

**Explanation**:
- The `ShardManager` routes queries based on the shard key (here `customerId`).
- For queries that lack the shard key (`findById` without `customerId`), we must query all shards (scatter‑gather) which is expensive. This is a key design consideration: choose shard key to match your common query patterns.

### Partitioning (Within a Database)

**Partitioning** is similar to sharding but occurs within a single database instance (e.g., PostgreSQL table partitioning). It splits a table into smaller physical pieces while keeping a logical view.

### Trade‑offs and Considerations

- **Shard Key Selection** – Critical for performance. Must evenly distribute data and support common query patterns.
- **Cross‑Shard Transactions** – Not supported; use sagas.
- **Resharding** – Plan for growth. Consistent hashing can help minimize data movement.
- **Joins** – Avoid joins across shards; denormalize or use application‑level joins.

### Sharding in Practice

Many databases offer built‑in sharding (e.g., MongoDB sharding, Cassandra partitioning). In relational databases, you often implement sharding at the application level or use middleware (e.g., Vitess for MySQL, Citus for PostgreSQL).

---

## Chapter Summary

This chapter explored four essential patterns for managing data in distributed systems:

1. **CQRS** separates write and read models, allowing each to be optimised and scaled independently. It introduces eventual consistency but provides flexibility for complex queries and high‑performance reads.

2. **Event Sourcing** stores state as a sequence of immutable events. It enables auditability, time travel, and flexible projections, but adds complexity and requires careful versioning.

3. **Saga** manages distributed transactions across services using local transactions and compensating actions. It can be choreographed via events or orchestrated by a central coordinator.

4. **Sharding** partitions data across multiple databases to achieve scalability. Choosing the right shard key and strategy is critical, and cross‑shard queries must be handled carefully.

**Key Insight**: In distributed systems, we trade strong consistency and simplicity for scalability and resilience. These patterns provide the tools to make that trade‑off successfully.

---

## Next Chapter Preview

**Chapter 18: Integration and Scalability (Strangler Fig, Sidecar, API Gateway, Leader Election)**

The final chapter in Part V will cover patterns that help integrate systems and scale them effectively. We'll explore the **Strangler Fig pattern** for gradually migrating legacy systems, the **Sidecar pattern** for offloading common tasks, the **API Gateway** as a single entry point, and **Leader Election** for coordinating distributed processes. These patterns complete the toolkit for building robust, scalable, and evolvable distributed systems.



<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='16. resilience_and_fault_tolerance.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='18. integration_and_scalability.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
